In [1]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
def energia_por_trimestre(X,variable):
    return X.groupby([variable]).sum()

In [3]:
data_dir_path = os.path.join('..','..','dataset','Series','Generacion')

data_list = list()
for file_name in os.listdir(data_dir_path):
    if 'HIDRAULICA' in file_name:
        file = pd.read_excel(os.path.join(data_dir_path,file_name))
        data_list.append(file)
        print(os.path.join(data_dir_path,file_name),file.shape)

..\..\dataset\Series\Generacion\HIDRAULICA_0.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_1.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_10.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_11.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_12.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_13.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_14.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_2.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_3.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_4.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_5.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_6.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_7.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_8.xlsx (1044040, 10)
..\..\dataset\Series\Generacion\HIDRAULICA_9.xlsx (1044040, 10)


In [8]:
full_data = pd.concat(data_list)
filter_df = full_data[full_data['Fecha']>='2018-01-01']

In [9]:
data_por_trimestre = filter_df.groupby(pd.Grouper(key="Fecha", freq="3M")).apply(lambda x: energia_por_trimestre(x,'Recurso'))
trimestres = pd.unique(data_por_trimestre.index.get_level_values('Fecha'))

In [10]:
variable_general = 'Generación'
variable_especifica = 'Recurso'

In [11]:
datos = data_por_trimestre.copy()
intervalos_dict = dict()
cuantiles = np.linspace(0.5,0.95, num=10)

for cuantil in cuantiles:

    intervalos_dict[cuantil] = dict()

    for trimestre in trimestres:

        trimestre_filtrado = datos.iloc[datos.index.get_level_values('Fecha') == trimestre]
        trimestre_filtrado = trimestre_filtrado.reset_index()

        labels = pd.unique(trimestre_filtrado[variable_especifica])

        trimestre_filtrado.sort_values(by=['kWh'],inplace=True,ascending=False)

        distribucion_quantiles = trimestre_filtrado['kWh'].quantile([cuantil])

        mascara_cuantil = trimestre_filtrado['kWh']>=distribucion_quantiles[cuantil]
        superior_al_cuantil = trimestre_filtrado[mascara_cuantil]
        
        for recurso in pd.unique(superior_al_cuantil['Recurso']):
            if recurso in intervalos_dict[cuantil]:
                intervalos_dict[cuantil][recurso] += 1
            else:
                intervalos_dict[cuantil][recurso] = 1    

In [19]:
ocurrencia_recurso_cuantil = pd.DataFrame(intervalos_dict)
ocurrencia_recurso_cuantil.fillna(0,inplace=True)

In [20]:
ocurrencia_recurso_cuantil = ocurrencia_recurso_cuantil.sort_values(by=list(cuantiles[::-1]),ascending=False)

In [23]:
ocurrencia_recurso_cuantil.head(50)

,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95
SAN CARLOS,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
GUAVIO,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
PORCE,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
SOGAMOSO,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
PAGUA,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,8.0
CHIVOR,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,9.0,8.0
GUATAPE,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0
GUATRON,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0
BETANIA,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
EL QUIMBO,10,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
